In [ ]:
import os
import matplotlib.pylab as plt
from glob import glob
import numpy as np

In [ ]:
PATH = os.path.abspath('../data')

SOURCE_IMAGES = [os.path.join(PATH, "images/um")]

images = []
labels = []

for si in SOURCE_IMAGES:
    images.extend(glob(os.path.join(si, "*.png")))
    labels.extend(glob(os.path.join(si.replace("images/","labels/egolane/"), "*.png")))
    
print("Datasets used: {}\n".format(SOURCE_IMAGES))

images.sort()
labels.sort()

In [ ]:
print(len(images))
print(len(labels))

In [ ]:
def get_image(path):
    
    image = plt.imread(path, 0)
    
    return(np.asarray(image[:320,:1152]))

In [ ]:
def get_label(path):

    label = plt.imread(path, 0)
    
    return(np.asarray(label[:320,:1152]))

In [ ]:
for i in images:
    print(get_image(i).shape)

In [ ]:
img = get_image(images[0])
img.dtype

In [ ]:
print(get_image(images[83]).shape)
print(get_image(labels[84]).shape)

In [ ]:
lbl = get_label(labels[0])
lbl.dtype
lbl.shape[:2]

In [ ]:
plt.imshow(lbl)

In [ ]:
def rgbto2(lbl):
    w,h = lbl.shape[:2]
    out = np.zeros([w,h],dtype=np.uint8)
    for i in range(w):
        for j in range(h):
            if(lbl[i,j,2] == 255):
                out[i,j] = 1
    return out

In [ ]:
out = rgbto2(lbl)
out.shape

In [ ]:
plt.imshow(out)

In [1]:
import random
import sys
import os
from optparse import OptionParser
import numpy as np
from PIL import Image

In [2]:
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch import optim

In [3]:
from eval import eval_net
from unet import UNet

In [4]:
def get_ids(directory):
    """Returns a list of the ids in the directory"""
    return (f[:-4] for f in os.listdir(directory))

In [5]:
def split_ids(ids, n=2):
    """Split each id in n, creating n tuples (id, k) for each id"""
    return ((iid, i) for i in range(n) for iid in ids)

In [6]:
def resize_and_crop(pilimg, scale=0.5, final_height=None):
    w = pilimg.size[0]
    h = pilimg.size[1]
    newW = int(w * scale)
    newH = int(h * scale)

    if not final_height:
        diff = 0
    else:
        diff = newH - final_height

    img = pilimg.resize((newW, newH))
    img = img.crop((0, diff // 2, newW, newH - diff // 2))
    return np.array(img, dtype=np.float32)

In [7]:
def get_square(img, pos):
    """Extract a left or a right square from ndarray shape : (H, W, C))"""
    h = img.shape[0]
    if pos == 0:
        return img[:, :h]
    else:
        return img[:, -h:]

In [254]:
def to_cropped_imgs(ids, directory, suffix, scale):
    """From a list of tuples, returns the correct cropped img"""
    for iid, pos in ids:
        im = resize_and_crop(Image.open(directory + iid + suffix), scale=scale)
        yield get_square(im, pos)

In [286]:
def load_img(ids, directory, suffix):
    for iid in ids:
        img = np.array(Image.open(directory + iid + suffix))
        yield img[:320,:1152]

In [ ]:
np.array(Image.open(directory + iid + suffix))

In [256]:
def load_mask(ids, directory, suffix):
    for iid in ids:
        msk = np.array(Image.open(directory + iid + suffix)).astype('int32')
        yield msk[:320,:1152]

In [257]:
def hwc_to_chw(img):
    return np.transpose(img, axes=[2, 0, 1])

In [258]:
def normalize(x):
    return np.float32(x/255.)

In [259]:
def rgbto2(lbl):
    w,h = lbl.shape[:2]
    out = np.zeros([w,h],dtype=np.uint8)
    for i in range(w):
        for j in range(h):
            if(lbl[i,j,2] == 255):
                out[i,j] = 1
    return out

In [260]:
def get_imgs_and_masks(ids, dir_img, dir_mask, scale):
    """Return all the couples (img, mask)"""

    imgs = to_cropped_imgs(ids, dir_img, '.png', scale)

    # need to transform from HWC to CHW
    imgs_switched = map(hwc_to_chw, imgs)
    imgs_normalized = map(normalize, imgs_switched)

    masks = to_cropped_imgs(ids, dir_mask, '.png', scale)

    return zip(imgs_normalized, masks)

In [261]:
def get_imgs_and_masks2(ids, dir_img, dir_mask):
    """Return all the couples (img, mask)"""

    imgs = load_img(ids, dir_img, '.png')

    # need to transform from HWC to CHW
    imgs_switched = map(hwc_to_chw, imgs)
    imgs_normalized = map(normalize, imgs_switched)

    masks = load_mask(ids, dir_mask, '.png')

    return zip(imgs_normalized, masks)

In [262]:
def get_full_img_and_mask(iid, dir_img, dir_mask):
    im = Image.open(dir_img + iid + '.png')
    mask = Image.open(dir_mask + iid + '.png')
    return np.array(im), np.array(mask)

In [263]:
def batch(iterable, batch_size):
    """Yields lists by batch"""
    b = []
    for i, t in enumerate(iterable):
        b.append(t)
        if (i + 1) % batch_size == 0:
            yield b
            b = []

    if len(b) > 0:
        yield b

In [264]:
def split_train_val(dataset, val_percent=0.05):
    dataset = list(dataset)
    length = len(dataset)
    n = int(length * val_percent)
    random.shuffle(dataset)
    return {'train': dataset[:-n], 'val': dataset[-n:]}

In [18]:
net = UNet(n_channels=3, n_classes=1)

In [19]:
epochs = 1
batch_size = 1
lr = 0.1
val_percent = 0.25
save_cp = True
gpu = False

In [20]:
dir_img = '../data/images/um/'
dir_mask = '../data/labels/egolane/um/'
dir_checkpoint = 'checkpoints/'

In [21]:
dir_img

'../data/images/um/'

In [32]:
ids = get_ids(dir_img)

In [31]:
len(list(ids.copy)

AttributeError: 'generator' object has no attribute 'copy'

In [33]:
iddataset = split_train_val(ids, val_percent)

In [72]:
iddataset['train']

['um_000022',
 'um_000061',
 'um_000045',
 'um_000000',
 'um_000050',
 'um_000058',
 'um_000067',
 'um_000008',
 'um_000005',
 'um_000043',
 'um_000073',
 'um_000023',
 'um_000027',
 'um_000064',
 'um_000038',
 'um_000037',
 'um_000044',
 'um_000006',
 'um_000052',
 'um_000083',
 'um_000077',
 'um_000088',
 'um_000020',
 'um_000062',
 'um_000093',
 'um_000091',
 'um_000004',
 'um_000002',
 'um_000068',
 'um_000060',
 'um_000007',
 'um_000051',
 'um_000035',
 'um_000001',
 'um_000028',
 'um_000015',
 'um_000082',
 'um_000087',
 'um_000024',
 'um_000086',
 'um_000025',
 'um_000049',
 'um_000085',
 'um_000009',
 'um_000048',
 'um_000013',
 'um_000040',
 'um_000054',
 'um_000012',
 'um_000018',
 'um_000047',
 'um_000033',
 'um_000016',
 'um_000070',
 'um_000066',
 'um_000019',
 'um_000032',
 'um_000026',
 'um_000057',
 'um_000081',
 'um_000003',
 'um_000021',
 'um_000029',
 'um_000069',
 'um_000046',
 'um_000042',
 'um_000053',
 'um_000079',
 'um_000034',
 'um_000092',
 'um_000075',
 'um_0

In [35]:
print('''
Starting training:
Epochs: {}
Batch size: {}
Learning rate: {}
Training size: {}
Validation size: {}
Checkpoints: {}
CUDA: {}
'''.format(epochs, batch_size, lr, len(iddataset['train']),
       len(iddataset['val']), str(save_cp), str(gpu)))


Starting training:
Epochs: 1
Batch size: 1
Learning rate: 0.1
Training size: 72
Validation size: 23
Checkpoints: True
CUDA: False



In [36]:
N_train = len(iddataset['train'])

In [37]:
N_train

72

In [38]:
N_val = len(iddataset['val'])

In [39]:
N_val

23

In [40]:
optimizer = optim.SGD(net.parameters(),
                  lr=lr,
                  momentum=0.9,
                  weight_decay=0.0005)

In [41]:
optimizer = optim.Adam(net.parameters(), lr=lr)

In [42]:
criterion = nn.BCELoss()

In [214]:
train = get_imgs_and_masks2(iddataset['train'], dir_img, dir_mask)

In [203]:
train

In [150]:
for i, (element1, element2) in enumerate(train):
    print(element1, element2)

In [139]:
element1.shape

(3, 375, 1242)

In [143]:
element1.dtype

dtype('float32')

In [144]:
type(element1)

numpy.ndarray

In [145]:
element2.shape

(375, 1242)

In [146]:
element2.dtype

dtype('int8')

In [151]:
type(element2)

numpy.ndarray

In [283]:
train = get_imgs_and_masks2(iddataset['train'], dir_img, dir_mask)

In [284]:
for i, b in enumerate(batch(train, batch_size=16)):
        imgs = np.array([i[0] for i in b])
        print(imgs.dtype)
        true_masks = np.array([i[1] for i in b])
        print(true_masks.dtype)
        imgs = torch.from_numpy(imgs)
        true_masks = torch.from_numpy(true_masks)

float32
int32
float32
int32
float32
int32
float32
int32
float32
int32


In [270]:
imgs.shape

torch.Size([2, 3, 320, 1152])

In [271]:
true_masks.shape

torch.Size([2, 320, 1152])

In [ ]:
for epoch in range(epochs):
    print('Starting epoch {}/{}.'.format(epoch + 1, epochs))
    net.train()

    # reset the generators
    train = get_imgs_and_masks2(iddataset['train'], dir_img, dir_mask)
    val = get_imgs_and_masks2(iddataset['val'], dir_img, dir_mask)

    epoch_loss = 0

    for i, b in enumerate(batch(train, batch_size)):
        imgs = np.array([i[0] for i in b]).astype(np.float32)
        true_masks = np.array([i[1] for i in b])

        imgs = torch.from_numpy(imgs)
        true_masks = torch.from_numpy(true_masks)

        if gpu:
            imgs = imgs.cuda()
            true_masks = true_masks.cuda()

        masks_pred = net(imgs)
        masks_probs_flat = masks_pred.view(-1)

        true_masks_flat = true_masks.view(-1)

        loss = criterion(masks_probs_flat, true_masks_flat)
        epoch_loss += loss.item()

        print('{0:.4f} --- loss: {1:.6f}'.format(i * batch_size / N_train, loss.item()))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch finished ! Loss: {}'.format(epoch_loss / i))

    if 1:
        val_dice = eval_net(net, val, gpu)
        print('Validation Dice Coeff: {}'.format(val_dice))

    if save_cp:
        torch.save(net.state_dict(),
                   dir_checkpoint + 'CP{}.pth'.format(epoch + 1))
        print('Checkpoint {} saved !'.format(epoch + 1))